In [1]:
import pandas as pd
import numpy as np
import math
import pickle

# Functions

In [2]:
PI = math.pi

def fix_units(df, inplace = False):
    
    if not inplace:
        ball = df.copy()
    else:
        ball = df
        
    ball.theta *= PI/180 # Degrees to Radians
    ball.phi *= PI/180 #Degrees to Radians

    ball.launch_speed *= 0.44704  # mile/h --> m/s
    ball.side_spin /= 60/(2*PI) # rpm to rads per s
    ball.back_spin /= 60/(2*PI) # rpm to rads per s
    if not inplace:
        return ball

In [3]:
def get_pred(input_dict, reg_carry,reg_tot_dist, reg_offline_ratio):
    
    mod_input = {s:[input_dict[s]] for s in input_dict}
    df = pd.DataFrame(mod_input)
    df.columns = ["launch_speed", "theta", "phi", "side_spin", "back_spin"]
    fix_units(df, inplace = True)

    df["cos_theta"] = df.theta.apply(lambda x: math.cos(x))
    df["sin_theta"] = df.theta.apply(lambda x: math.sin(x))
    df["tan_theta"] = df.theta.apply(lambda x: math.tan(x))

    df["cos_phi"] = df.phi.apply(lambda x: math.cos(x))
    df["sin_phi"] = df.phi.apply(lambda x: math.sin(x))
    df["tan_phi"] = df.phi.apply(lambda x: math.tan(x))

    cols = list(df.columns)
    for i,col in enumerate(cols):
        rest = cols[:i]+cols[i+1:]
        for r_col in rest:
            df[col+"_"+r_col] = df[col]*df[r_col]

    return (reg_carry.predict(df), reg_tot_dist.predict(df), reg_tot_dist.predict(df)*reg_offline_ratio.predict(df))

# Retreiving models

In [5]:
with open('../models/reg_carry.pkl', 'rb') as f:
    reg_carry = pickle.load(f)

with open('../models/reg_tot_dist.pkl', 'rb') as f:
    reg_tot_dist = pickle.load(f)

with open('../models/reg_offline_ratio.pkl', 'rb') as f:
    reg_offline_ratio = pickle.load(f)

# Sample input

In [7]:
data = pd.read_excel("all_sessions_refined.xlsx")

In [8]:
data.head()

,Session,Club,Ball Speed,Launch Angle,Azimuth,Side Spin,Back Spin,Carry,Carry Offline,Total Distance,Offline
0,2,Driver,43.3,30.2,-3.1,-6943,7471,32.95,-5.758168,37.71,-6.59
1,2,Driver,44.1,31.3,-1.2,-2616,8297,35.79,-2.736265,40.94,-3.13
2,1,4h,50.9,18.9,-1.5,210,667,36.54,-0.219139,65.03,-0.39
3,2,Driver,49.4,58.6,-25.1,-499,1988,37.44,-17.060597,42.53,-19.38
4,2,Driver,50.3,59.3,-22.3,-770,2129,37.67,-16.117733,42.56,-18.21


In [9]:
data.columns = ["session", "club", "ball_speed", "launch_angle", "azimuth", "side_spin", 
                "back_spin", "carry", "carry_offline", "tot_dist", "offline"]

In [10]:
samp_input = data.iloc[0,:][["ball_speed", "launch_angle", "azimuth", "side_spin", "back_spin"]].to_dict()

In [11]:
samp_input

{'ball_speed': 43.3,
 'launch_angle': 30.2,
 'azimuth': -3.1,
 'side_spin': -6943,
 'back_spin': 7471}

# Prediction

In [12]:
carry_pred, tot_dist_pred, offline_pred = get_pred(samp_input, reg_carry,reg_tot_dist, reg_offline_ratio)

In [13]:
print("carry_pred:",carry_pred)
print("tot_dist_pred:",tot_dist_pred)
print("offline_pred:",offline_pred)

carry_pred: [33.01037759]
tot_dist_pred: [38.4794839]
offline_pred: [-7.1789683]


In [14]:
data.iloc[0,:][["carry","tot_dist","offline"]]

carry       32.95
tot_dist    37.71
offline     -6.59
Name: 0, dtype: object